# Red Neuronal y Procesamiento de Lenguaje Natural

In [23]:
# Librerias necesarias
import os, glob

import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.data import Dataset
from tensorflow.keras import losses
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Embedding, GRU, Dense

## 1. Crear una losya de palabras a generar

In [24]:
data = pd.read_csv('Datasets/nombres.csv')
data

,Nombre
0,Alejandro
1,Valeria
2,Mateo
3,Sofía
4,Daniel
...,...
95,Salma
96,Simón
97,Luna
98,Thiago


In [25]:
# Unir los nombres en un parrafo
text = ' '.join([data.iloc[i, 0].lower() for i in data.index])
text

'alejandro valeria mateo sofía daniel camila andrés isabella sebastián mariana diego natalia gabriel fernanda samuel lucía leonardo victoria emiliano regina rodrigo ximena julián paulina nicolás renata ángel carolina tomás alejandra benjamín jimena jorge elisa adrián mónica luis gabriela ricardo andrea bruno claudia mauricio laura alan patricia iván teresa óscar beatriz aarón cecilia esteban silvia raúl rosa enrique julia marcos ángela rafael lourdes hugo fabiola sergio daniela arturo bianca eduardo pamela manuel araceli francisco verónica alonso miriam cristian karla joel yesenia martín rocío césar estrella israel danna kevin aitana jonathan zoe dante abril gael alma elías salma simón luna thiago aurora'

## 2. Preparar los datos y crear el vocabulario (preprocesamiento)

In [26]:
# Vocabulartio
vocab = sorted(set(text))
char2idx = {c:i for i, c in enumerate(vocab)}
idx2char = np.array(vocab)

print(f'Vocabulario: {len(vocab)} caracteres unicos')
print(f'Caracteres: {vocab}')

Vocabulario: 31 caracteres unicos
Caracteres: [' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', 'á', 'é', 'í', 'ó', 'ú']


In [27]:
# Ejemplo de nombre codificado
name = data.iloc[0, 0].lower()
print(f'Nombre: {name}')
print(f'Codificacion: {[char2idx[c] for c in name]}')


Nombre: alejandro
Codificacion: [1, 12, 5, 10, 1, 14, 4, 18, 15]


In [28]:
# Texto codificado
encoded = np.array([char2idx[c] for c in text])
print(encoded)

[ 1 12  5 10  1 14  4 18 15  0 22  1 12  5 18  9  1  0 13  1 20  5 15  0
 19 15  6 28  1  0  4  1 14  9  5 12  0  3  1 13  9 12  1  0  1 14  4 18
 27 19  0  9 19  1  2  5 12 12  1  0 19  5  2  1 19 20  9 26 14  0 13  1
 18  9  1 14  1  0  4  9  5  7 15  0 14  1 20  1 12  9  1  0  7  1  2 18
  9  5 12  0  6  5 18 14  1 14  4  1  0 19  1 13 21  5 12  0 12 21  3 28
  1  0 12  5 15 14  1 18  4 15  0 22  9  3 20 15 18  9  1  0  5 13  9 12
  9  1 14 15  0 18  5  7  9 14  1  0 18 15  4 18  9  7 15  0 23  9 13  5
 14  1  0 10 21 12  9 26 14  0 16  1 21 12  9 14  1  0 14  9  3 15 12 26
 19  0 18  5 14  1 20  1  0 26 14  7  5 12  0  3  1 18 15 12  9 14  1  0
 20 15 13 26 19  0  1 12  5 10  1 14  4 18  1  0  2  5 14 10  1 13 28 14
  0 10  9 13  5 14  1  0 10 15 18  7  5  0  5 12  9 19  1  0  1  4 18  9
 26 14  0 13 29 14  9  3  1  0 12 21  9 19  0  7  1  2 18  9  5 12  1  0
 18  9  3  1 18  4 15  0  1 14  4 18  5  1  0  2 18 21 14 15  0  3 12  1
 21  4  9  1  0 13  1 21 18  9  3  9 15  0 12  1 21

## 3. Crear datos para entrenar la red (secuencias)

In [29]:
seq_length = 16
char_dataset = Dataset.from_tensor_slices(encoded)
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    return chunk[:-1], chunk[1:]

dataset = sequences.map(split_input_target)

BATCH_SIZE = 32
BUFFER_SIZE = 1000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

## 4. Crear red neuronal recurrente

In [30]:
vocb_size = len(vocab)
embedding_dim = 128
rnn_units = 512

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    return Sequential([
        Input(batch_shape=(batch_size, None)),
        Embedding(vocab_size, embedding_dim),
        GRU(rnn_units, return_sequences=True, stateful=True, 
            recurrent_initializer='glorot_uniform'),
        GRU(rnn_units // 2, return_sequences=True, stateful=True, 
            recurrent_initializer='glorot_uniform'),
        Dense(vocab_size)
    ])

model = build_model(vocb_size, embedding_dim, rnn_units, BATCH_SIZE)

def loss(labels, logits):
    return losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

checkpoint_dir = './training_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch:020}.weights.h5")

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

EPOCHS = 100
model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 3.4338
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 3.4005
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 157ms/step - loss: 3.3483
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 135ms/step - loss: 3.2374
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 154ms/step - loss: 3.0324
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 143ms/step - loss: 2.9757
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - loss: 2.9903
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 166ms/step - loss: 2.8762
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 171ms/step - loss: 2.9083
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step - loss: 2.9321
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - loss: 2.9461
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step - loss: 2.9218
Epoch 13/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - loss: 2.8878
Epoch 14/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 161ms/step - loss: 2.8632
Epoch 15/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 184ms/step - loss: 2.8270
Epoch 1